# GenAI/RAG in Python 2025

## Session 03. 

## (1) Postgres+pgvector Vector Storage
## (2) Function Calling

In [ ]:
import os
import numpy as np
import pandas as pd
from openai import OpenAI

## 1. Postgres + pgvector as our vector storage

Install Docker image:

```
docker pull pgvector/pgvector:pg16
```

Run the container:

```
docker run --name ragdb \
  -e POSTGRES_USER=raguser \
  -e POSTGRES_PASSWORD=ragpass \
  -e POSTGRES_DB=ragdb \
  -p 5432:5432 \
  -d pgvector/pgvector:pg16
```

| Option                 | Meaning                                                 |
| ---------------------- | ------------------------------------------------------- |
| `--name ragdb`         | names the container (handy for stopping/starting later) |
| `-e POSTGRES_USER`     | creates a default Postgres user                         |
| `-e POSTGRES_PASSWORD` | sets that user’s password                               |
| `-e POSTGRES_DB`       | creates a database on startup                           |
| `-p 5432:5432`         | exposes port 5432 on localhost                          |
| `-d`                   | runs in detached (background) mode                      |


Postgres + pgvector is now running locally on port `5432`.

Verify:

```
docker exec -it ragdb psql -U raguser -d ragdb
```

```
CREATE EXTENSION IF NOT EXISTS vector;
\dx
```

```
SELECT 'pgvector ready' AS status;
```

Stopping / cleaning up

```
docker stop ragdb         # stop the container
docker start ragdb        # restart it
docker rm -f ragdb        # remove it completely
docker volume rm pgdata   # remove stored data (optional)
```

In [ ]:
df = pd.read_csv("_data/italian_recipes_embedded.csv")

In [ ]:
df

We want to migrate `df` to Postgres now... 

In [ ]:
df.dtypes

Note: `embedding` is an `object` (a string indeed).

In [ ]:
import ast
import psycopg
from psycopg.rows import dict_row

In [ ]:
# --- 1. Parse string embeddings into numpy arrays ---
df['embedding_vector'] = df['embedding'].apply(
    lambda x: np.array(ast.literal_eval(x), dtype=np.float32)
)
df

Why?

In [ ]:
df['embedding'][0]

In [ ]:
type(df['embedding'][0])

In [ ]:
list(df['embedding'][0])

Nope.

In [ ]:
np.array(ast.literal_eval(df['embedding'][0]), dtype=np.float32)

Because:

In [ ]:
ast.literal_eval(df['embedding'][0])

In [ ]:
type(ast.literal_eval(df['embedding'][0]))

Create Postgres table:

In [ ]:
# --- 1. Connect to Postgres ---
from pgvector.psycopg import register_vector
conn = psycopg.connect(
    host="localhost",
    dbname="ragdb",
    user="raguser",
    password="ragpass",
    port=5432
)

register_vector(conn)

cur = conn.cursor()

Drop the `receipts` table if the code was previously executed.

In [ ]:
cur.execute("DROP TABLE IF EXISTS receipts;")
conn.commit()

In [ ]:
# --- 2. Create table if not exists ---

# - dimensionality
dim = len(df['embedding_vector'][0])

# - create:
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS receipts (
    id SERIAL PRIMARY KEY,
    title TEXT,
    receipt TEXT,
    embedding VECTOR({dim})
);
"""
cur.execute(create_table_sql)
conn.commit()

Transfer:

In [ ]:
for _, row in df.iterrows():
    cur.execute(
        "INSERT INTO receipts (title, receipt, embedding) VALUES (%s, %s, %s)",
        (row['title'], row['receipt'], row['embedding_vector'].tolist())
    )

conn.commit()

Check table:

In [ ]:
# - a simple query to test
cur.execute("""
    SELECT id, title, receipt, embedding
    FROM receipts
    LIMIT 5;
""")

# - fetch results
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]

# DataFrame
df_test = pd.DataFrame(rows, columns=colnames)
df_test

### Client:

In [ ]:
# Set your API key (ensure OPENAI_API_KEY is set in your environment)
api_key = os.getenv("OPENAI_API_KEY")

# Instantiate the OpenAI client with your API key  
client = OpenAI(api_key=api_key)

In [ ]:
user_text = """
Hi! I’d like to cook a good Italian dish for lunch! I have potatoes, carrots, 
rosemary, and pork. Can you recommend a recipe and help me a bit with 
preparation tips?
"""

... and of course we need an embedding of `user_text` as well:

In [ ]:
# Select the embedding model to use (as per OpenAI docs)  
model_name = "text-embedding-3-small"   

resp = client.embeddings.create(        
        model=model_name,                   
        input=[user_text]                        
    )
user_query = resp.data[0].embedding

print(type(user_query))
print(len(user_query))

Find the most suitable examples that match the user input: Cosine Distance (Similarity) in Postgres

In [ ]:
# Run the retrieval query using pgvector operator
# NOTE: ::vector - the "vector adapter"

sql = """
SELECT id, title, receipt, 1 - (embedding <=> %s::vector) AS similarity
FROM receipts
ORDER BY embedding <=> %s::vector
LIMIT 5;
"""
cur.execute(sql, (user_query, user_query))

rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]

# - Load into pandas DataFrame
prompt_recipes = pd.DataFrame(rows, columns=colnames)

# Show results
print(prompt_recipes)

Nice; clean-up:

In [ ]:
cur.close()
conn.close()

Integrate results into the prompt:

In [ ]:
# Build a single output string with titles and recipes
output_lines = []
for _, row in prompt_recipes.iterrows():
    title = row["title"]
    recipe = row["receipt"]
    output_lines.append(f"{title}:\n{recipe}")
prompt_recipes = "\n\n".join(output_lines)
print(prompt_recipes)

In [ ]:
prompt = f"""
You are a helpful Italian cooking assistant.  
Here are some recipe examples I found that may or may not be relevant to the user's request:

{prompt_recipes}

User’s question: "{user_text}"

From the examples above:
1. Determine which recipes are *relevant* to what the user asked and which are not.
2. Discard or ignore irrelevant ones, and focus on relevant ones.
3. For each relevant example, rephrase the recipe in a more narrative, 
conversational style, adding cooking tips, alternative ingredients, variations, 
or suggestions.
4. Then produce a final response to the user: a narrative that weaves 
together those enhanced recipes (titles + steps + tips) in an engaging way.
5. Don't forget to use the original titles of the recipes.
6. Advise on more than one recipe - if there are more than one relevant!

Do not just list recipes — tell a story, connect to the user's question, 
and use the examples as inspirations, but enhance them.  
Make sure your response is clear, helpful, and focused on what the user wants.
"""

Run prompt, sit and enjoy:

In [ ]:
response = client.chat.completions.create(
    model="gpt-4",    # or whichever model you prefer
    messages=[
        {"role": "system", "content": "You are a helpful Italian cooking assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=5000
)

reply_text = response.choices[0].message.content
print(reply_text)

## 2. Function Calling with OpenAI ChatGPT

OpenAI's function calling feature allows ChatGPT to output structured data (like JSON) to call external functions or APIs. This means the model can decide when to use a function you provide, and it will return the function name and arguments it wants to call. The developer's code can then execute that function (e.g., call an external API) and pass the result back to the model. This is useful for retrieving real-time information (like weather or stock prices) or performing computations that the model can't do by itself.

### 2.1 The Tool

First, we need an external API that our tool (previously known as: `function` in the OpenAI API) will call. For this educational example, we'll use the `wttr.in` weather API. It's a free API (no API key needed) that returns current weather information for a given location in JSON format.

We'll write a Python function `get_current_weather(location)` that calls this API. This function will serve as the "tool" that ChatGPT can use via function calling. In a real application, this function might call any external service or perform some calculation. Here, it will fetch weather data and return a simple text summary.

In [ ]:
import requests

def get_current_weather(location: str) -> str:
    """
    Fetches the current weather in Celsius for a given location using wttr.in.
    Returns a simple summary string.
    """
    url = f"http://wttr.in/{location}?format=j1"
    data = requests.get(url).json()
    current = data["current_condition"][0]
    temp_c = current["temp_C"]
    desc = current["weatherDesc"][0]["value"]
    return f"Temperature: {temp_c}°C, Condition: {desc}"

### 2.2. Defining the Function Schema for the OpenAI API

Now that we have a Python function, we need to tell the OpenAI API about it. We do this by providing a tool schema in our API call. The schema includes the tool name, description, and parameters (with types) that the tool accepts. This information helps the model decide when and how to call the function.

We'll prepare a dictionary that follows the OpenAI function calling specification. This will later be passed to the functions parameter of the chat completion API call.

In [ ]:
# Define the function schema for the OpenAI API
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a city (Celsius only)",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g. Paris or London"
                    }
                },
                "required": ["location"]
            }
        }
    }
]


This schema is how we register the function with the model. With `functions=[...]` in the API call, GPT-3.5/4 is aware that it has an available tool named "get_current_weather" and how to call it.

In [ ]:
# messages
messages = [
    {"role": "user", 
     "content": "What's the weather in Paris right now?"}
]

# call
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0
)

# result
response_message = response.choices[0].message
print(response_message)

In [ ]:
resp = response_message.tool_calls[0].function.arguments
print(resp)

In [ ]:
print(type(resp))
import json
args = json.loads(resp)
print(type(args))

Find out:

In [ ]:
import json
weather_conditions = get_current_weather(args['location'])
print(weather_conditions)

Final response:

In [ ]:
messages[0]['content']

In [ ]:
prompt = f"""
This is the user formulated question: {messages[0]['content']}.

This is the response obtained from an weather API: {weather_conditions}.

Provide a polite response to the user in English including the info found 
in the weather API response and include some clothing recommendations.
"""

# messages
messages = [
    {"role": "user", 
     "content": prompt}
]

# call
response = client.chat.completions.create(
    messages = messages,
    model="gpt-4-1106-preview",
    temperature=0
)

In [ ]:
print(response.choices[0].message.content)